# 3. Training Pipeline

## 3.1. Setup

### 3.1.1. Import Libraries

In [1]:
# Standard imports
import os
from pathlib import Path
import sys
import json
import time
from datetime import date, datetime, timedelta
import warnings

warnings.filterwarnings("ignore", module="IPython")
warnings.filterwarnings("ignore", category=DeprecationWarning)

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import numpy as np
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError, Timeout, RequestException
from collections import defaultdict
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata, visualization

today = datetime.today().date()

Project root dir: c:\Users\krist\Documents\GitHub\pm25


### 3.1.2. Load Settings and Initialize Hopsworks Connection 

In [2]:
# ---------------------------------------------------------
# 1. Detect environment (local, Jupyter, or Hopsworks Job)
# ---------------------------------------------------------
# Check for Hopsworks-specific environment variables
RUNNING_IN_HOPSWORKS = (
    "HOPSWORKS_PROJECT_ID" in os.environ or 
    "HOPSWORKS_JOB_ID" in os.environ or
    "HOPSWORKS_PROJECT_NAME" in os.environ
)

if RUNNING_IN_HOPSWORKS:
    print("🔧 Running in Hopsworks environment")
    # Running inside Hopsworks → use implicit login and secrets vault
    project = hopsworks.login()
    secrets_api = hopsworks.get_secrets_api()

    os.environ["HOPSWORKS_API_KEY"] = secrets_api.get_secret("HOPSWORKS_API_KEY").value
    os.environ["AQICN_API_KEY"] = secrets_api.get_secret("AQICN_API_KEY").value
    os.environ["GH_PAT"] = secrets_api.get_secret("GH_PAT").value
    os.environ["GH_USERNAME"] = secrets_api.get_secret("GH_USERNAME").value
    
    # Now load settings (will read from os.environ we just set)
    settings = config.HopsworksSettings()
    
    HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
    AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()
    GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

else:
    print("💻 Running locally")
    # Running locally → use .env file
    from dotenv import load_dotenv
    load_dotenv()
    
    # Load settings from .env
    settings = config.HopsworksSettings()
    
    HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
    AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()
    GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()
    
    # Login to Hopsworks using the API key
    project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)

# ---------------------------------------------------------
# 2. Get feature store (works for both environments now)
# ---------------------------------------------------------
fs = project.get_feature_store()


💻 Running locally
HopsworksSettings initialized!
2026-01-26 18:41:10,058 INFO: Initializing external client
2026-01-26 18:41:10,058 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-26 18:41:12,790 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


### 3.1.3. Repository Management

In [3]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

💻 Already in git repository at c:\Users\krist\Documents\GitHub\pm25


### 3.1.4. Configure API Keys and Secrets

In [4]:
secrets = hopsworks.get_secrets_api()

try:
    secrets.get_secret("AQICN_API_KEY")
except:
    secrets.create_secret("AQICN_API_KEY", settings.AQICN_API_KEY.get_secret_value())

### 3.1.5. Get Model Registry

In [5]:
mr = project.get_model_registry()

## 3.2. Load Feature Groups & Metadata

In [6]:
air_quality_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

# Load data from air_quality feature group
aq_data = air_quality_fg.read()

if len(aq_data) == 0:
    print("⚠️ No air quality data found. Run pipeline 1 (backfill) first.")
    sys.exit(1)

# Extract metadata and set sensor_id as index for lookups
aq_metadata = aq_data[["sensor_id", "latitude", "longitude", "city", "street", "country"]].drop_duplicates(subset=["sensor_id"]).set_index("sensor_id")

# Build sensor location dictionary
sensor_locations = {}
for sensor_id, row in aq_metadata.iterrows():
    sensor_locations[sensor_id] = (
        row["latitude"], 
        row["longitude"], 
        row["city"], 
        row["street"], 
        row["country"]
    )
print(f"📍 Loaded locations for {len(sensor_locations)} sensors")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (84.25s) 
📍 Loaded locations for 103 sensors


## 3.3. Create Additional Feature Views
Create multiple feature views with different feature combinations (baseline, rolling windows, lagged features, nearby sensors, complete) for model comparison.

In [7]:
# Select features for training data
baseline_features = air_quality_fg.select(["pm25", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])

baseline_feature_view = fs.get_or_create_feature_view(
    name="air_quality_baseline_fv",
    description="Weather features for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=baseline_features,
)

rolling_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
rolling_feature_view = fs.get_or_create_feature_view(
    name="air_quality_rolling_fv",
    description="Weather features, PM2.5 rolling window (3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=rolling_features,
)

nearby_features = air_quality_fg.select(["pm25", "pm25_nearby_avg", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
nearby_feature_view = fs.get_or_create_feature_view(
    name="air_quality_nearby_fv",
    description="Weather features, PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=nearby_features,
)

lagged_1d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_1d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_1d_fv",
    description="Weather features, PM2.5 lags (1d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_1d_features,
)

lagged_2d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_2d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_2d_fv",
    description="Weather features, PM2.5 lags (1d, 2d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_2d_features,
)

lagged_3d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_3d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_3d_fv",
    description="Weather features, PM2.5 lags (1d, 2d, 3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_3d_features,
)

complete_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "pm25_nearby_avg", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
complete_feature_view = fs.get_or_create_feature_view(
    name="air_quality_complete_fv",
    description="Weather features, PM2.5 rolling window (3d), and PM2.5 lags (1d, 2d, 3d), and PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=complete_features,
)

2026-01-26 18:42:44,005 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-26 18:42:46,593 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-26 18:42:48,608 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-26 18:42:50,594 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-26 18:42

## 3.4. Training Setup

### 3.4.1. Define Training Directory

In [8]:
model_dir = f"{root_dir}/models"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    
print(f"Model directory: {model_dir}")

Model directory: c:\Users\krist\Documents\GitHub\pm25/models


### 3.4.2. Define Model Hyperparameters

In [9]:
TARGET = "pm25"

TRAIN_RATIO = 0.8
MIN_ROWS = 10
MIN_TEST_ROWS = 2

EXCLUDE_COLS = [
    "pm25","date","sensor_id","city","street","country",
    "latitude","longitude","aqicn_url"
]

N_RESTARTS = 5
BASE_SEED = 165439

xgb_params = {
    "n_estimators": 100,
    "learning_rate": 0.05,
}

### 3.4.3. Initialize Containers for Results

In [10]:
# Prepare containers to store models, predictions, and results
models = defaultdict(dict)
y_preds = defaultdict(dict)
results = []

# Define feature view dictionary for iteration
feature_views = {
    "baseline": baseline_feature_view,
    "rolling": rolling_feature_view,
    "nearby": nearby_feature_view,
    "lagged_1d": lagged_1d_feature_view,
    "lagged_2d": lagged_2d_feature_view,
    "lagged_3d": lagged_3d_feature_view,
    "complete": complete_feature_view,
}

## 3.5. Training Loop
Train XGBoost models for each feature combination and sensor, run 5 iterations per configuration, select best model based on R2 score, and store results.

### 3.5.1. Load Feature View Data

In [12]:
feature_data_cache = {}
total_views = len(feature_views)

for i, (feature_name, feature_view) in enumerate(feature_views.items(), start=1):
    print(f"Reading ({i}/{total_views}): {feature_name}")

    df = feature_view.query.read()
    df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)
    feature_data_cache[feature_name] = df

    print(f"    ✔ Loaded {len(df):,} rows")

print(f"\n✅ Loaded all {total_views} feature views")

Reading (1/7): baseline
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (34.25s) 
    ✔ Loaded 165,668 rows
Reading (2/7): rolling
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (44.33s) 
    ✔ Loaded 165,668 rows
Reading (3/7): nearby
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (37.43s) 
    ✔ Loaded 165,668 rows
Reading (4/7): lagged_1d
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (38.49s) 
    ✔ Loaded 165,668 rows
Reading (5/7): lagged_2d
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (36.75s) 
    ✔ Loaded 165,668 rows
Reading (6/7): lagged_3d
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (48.59s) 
    ✔ Loaded 165,668 rows
Reading (7/7): complete
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1417.20s) 
    ✔ Loaded 165,668 rows

✅ Loaded all 7 feature views


### 3.5.2. Build Task List

In [13]:
tasks = []
total_views = len(feature_data_cache)

print(f"Building task list from {total_views} feature views:")

for i, (feature_name, df) in enumerate(feature_data_cache.items(), start=1):
    sensor_ids = df["sensor_id"].unique()
    count = len(sensor_ids)

    print(f"[{i}/{total_views}] {feature_name}: {count} sensors")

    for sensor_id in sensor_ids:
        tasks.append((feature_name, sensor_id))

print(f"\n✅ {len(tasks):,} total training tasks")

Building task list from 7 feature views:
[1/7] baseline: 103 sensors
[2/7] rolling: 103 sensors
[3/7] nearby: 103 sensors
[4/7] lagged_1d: 103 sensors
[5/7] lagged_2d: 103 sensors
[6/7] lagged_3d: 103 sensors
[7/7] complete: 103 sensors

✅ 721 total training tasks


### 3.5.3. Training Loop

In [15]:
total = len(tasks)

for idx, (feature_name, sensor_id) in enumerate(tasks, start=1):
    df = feature_data_cache[feature_name]
    df = df[df["sensor_id"] == sensor_id].dropna(subset=[TARGET])

    if len(df) < MIN_ROWS:
        continue

    feature_cols = [c for c in df.columns if c not in EXCLUDE_COLS]

    train_size = int(TRAIN_RATIO * len(df))
    train_df = df.iloc[:train_size]
    test_df = df.iloc[train_size:]

    if len(test_df) < MIN_TEST_ROWS:
        continue

    X_train = train_df[feature_cols]
    y_train = train_df[TARGET]
    X_test = test_df[feature_cols]
    y_test = test_df[TARGET]

    best_model = None
    best_r2 = -1e9
    best_mse = 1e9

    for i in range(N_RESTARTS):
        model = XGBRegressor(
            n_estimators=xgb_params["n_estimators"],
            learning_rate=xgb_params["learning_rate"],
            random_state=BASE_SEED*i
        )
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        r2 = r2_score(y_test, pred)
        mse = mean_squared_error(y_test, pred)

        if r2 > best_r2:
            best_r2 = r2
            best_mse = mse
            best_model = model

    # Store results
    models.setdefault(feature_name, {})[sensor_id] = best_model
    y_preds.setdefault(feature_name, {})[sensor_id] = best_model.predict(X_test)
    results.append({
        "feature_name": feature_name,
        "sensor_id": sensor_id,
        "R2": best_r2,
        "MSE": best_mse,
        "train_size": len(train_df),
        "test_size": len(test_df),
    })

    if idx % 10 == 0 or idx == total:
        print(f"[{idx}/{total}] Trained {feature_name} / sensor {sensor_id}: R²={best_r2:.3f}, MSE={best_mse:.2f}")

print(f"\n✅ Training complete: {len(results)} models trained")

2026-01-26 20:16:05,312 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-26 20:16:05,314 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-26 20:16:05,316 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-26 20:16:05,316 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-26 20:16:05,421 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-26 20:16:05,425 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

## 3.6. Model Selection

### 3.6.1. Identify Best Model per Sensor

In [16]:
results_df = pd.DataFrame(results)

best_models = results_df.loc[results_df.groupby("sensor_id")["R2"].idxmax()]
best_models = best_models.set_index("sensor_id")

print(f"Identified best models for {len(best_models)} sensors:\n")

summary = best_models[["feature_name", "R2", "MSE"]].sort_index()

print(summary.to_string())


Identified best models for 103 sensors:

          feature_name        R2         MSE
sensor_id                                   
57421        lagged_1d  0.426792    4.549678
58666        lagged_1d  0.192240    3.970698
58909        lagged_2d  0.312698    7.284364
58912        lagged_1d  0.366746    5.499997
58921         complete  0.919423    0.043539
59095        lagged_1d  0.358960    9.549328
59356           nearby  0.127710   11.657171
59410        lagged_3d  0.278112    0.249105
59497        lagged_1d  0.455935    5.276114
59593        lagged_3d  0.218126    4.166982
59650        lagged_2d  0.362254    6.647994
59656        lagged_2d  0.069853  106.271086
59887        lagged_1d  0.253564   10.299361
59893        lagged_1d  0.621017    1.097945
59899        lagged_1d  0.371269    7.099150
60073        lagged_2d  0.508769    2.728570
60076        lagged_3d  0.337055   13.669023
60535        lagged_1d  0.203664   16.919514
60541         complete  0.297254   11.286639
60838        l

### 3.6.2. Load Feature View Data

In [17]:
total_views = len(feature_views)
print(f"Loading {total_views} feature views...\n")

cached_feature_data = {}

for i, (name, fv) in enumerate(feature_views.items(), start=1):
    print(f"[{i}/{total_views}] Reading feature view: {name}...")
    df_cached = fv.query.read()
    cached_feature_data[name] = df_cached
    print(f"    ✔ Loaded {len(df_cached):,} rows")

print("\nNormalizing date columns...\n")

for name, df_cached in cached_feature_data.items():
    df_cached["date"] = pd.to_datetime(df_cached["date"]).dt.tz_localize(None)
    print(f"    ✔ Normalized dates for '{name}'")

print("\nAll feature views loaded and normalized")


Loading 7 feature views...

[1/7] Reading feature view: baseline...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (45.73s) 
    ✔ Loaded 165,668 rows
[2/7] Reading feature view: rolling...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (36.63s) 
    ✔ Loaded 165,668 rows
[3/7] Reading feature view: nearby...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (24.88s) 
    ✔ Loaded 165,668 rows
[4/7] Reading feature view: lagged_1d...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (30.48s) 
    ✔ Loaded 165,668 rows
[5/7] Reading feature view: lagged_2d...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (83.67s) 
    ✔ Loaded 165,668 rows
[6/7] Reading feature view: lagged_3d...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (57.60s) 
    ✔ Loaded 165,668 rows
[7/7] Reading feature view: complete...
Finished: Read

### 3.9.3. Generate Predictions

In [18]:
all_test_data = []
total_sensors = len(best_models)

print(f"Processing {total_sensors} sensors...\n")

for idx, (_, row) in enumerate(best_models.iterrows(), start=1):
    sensor_id = row.name
    best_feature = row["feature_name"]

    status = []  # collect short status flags

    # --- Save model + feature importance ---
    sensor_dir = f"{model_dir}/{sensor_id}"
    images_dir = f"{sensor_dir}/images"
    os.makedirs(images_dir, exist_ok=True)

    best_model = models[best_feature][sensor_id]
    best_model.save_model(f"{sensor_dir}/model.json")
    plot_importance(best_model)
    plt.savefig(f"{images_dir}/feature_importance.png")
    plt.close()
    status.append("model+plot")

    # --- Load cached feature view data ---
    df = cached_feature_data[best_feature]
    df = df[df["sensor_id"] == sensor_id].copy()
    df = df.sort_values("date").reset_index(drop=True)
    df = df.dropna(subset=["pm25"])

    feature_cols = [
        c for c in df.columns
        if c not in ["pm25", "date", "sensor_id", "city", "street",
                     "country", "latitude", "longitude", "aqicn_url"]
    ]

    df_clean = df.copy()
    if len(df_clean) < 10:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (too few rows)")
        continue

    # --- Full predictions ---
    X_full = df_clean[feature_cols]
    df_clean["predicted_pm25"] = best_model.predict(X_full)
    df_clean["best_model"] = best_feature
    status.append("predictions")

    # --- Hindcast window ---
    cutoff_date = pd.Timestamp.now() - pd.DateOffset(months=18)
    df_hindcast = df_clean[df_clean["date"] >= cutoff_date].copy()

    if len(df_hindcast) == 0:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no recent data)")
        continue
    status.append("hindcast")
  
    # --- Metadata ---
    if sensor_id in aq_metadata.index:
        df_clean["latitude"] = aq_metadata.at[sensor_id, "latitude"]
        df_clean["longitude"] = aq_metadata.at[sensor_id, "longitude"]
        df_clean["city"] = aq_metadata.at[sensor_id, "city"]
        df_clean["street"] = aq_metadata.at[sensor_id, "street"]
        df_clean["sensor_id"] = sensor_id
        status.append("metadata")
    else:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no metadata)")
        continue

    # --- Append final data ---
    all_test_data.append(
        df_clean[[
            "date", "sensor_id", "pm25", "predicted_pm25",
            "latitude", "longitude", "city", "street", "best_model"
        ]]
    )
    status.append("saved")

    # --- One-line summary ---
    print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: " + ", ".join(status))

print(f"\nDone. Successfully processed {len(all_test_data)} sensors.")

Processing 103 sensors...

2026-01-26 20:58:43,069 WARNING: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.

[1/103] Sensor 57421: model+plot, predictions, hindcast, metadata, saved
[2/103] Sensor 58666: model+plot, predictions, hindcast, metadata, saved
[3/103] Sensor 58909: model+plot, predictions, hindcast, metadata, saved
[4/103] Sensor 58912: skipped (no recent data)
[5/103] Sensor 58921: model+plot, predictions, hindcast, metadata, saved
[6/103] Sensor 59095: model+plot, predictions, hindcast, metadata, saved
[7/103] Sensor 59356: model+plot, predictions, hindcast, metadata, saved
[8/103] Sensor 59410: model+plot, predictions, hindcast, metadata, saved
[9/103] Sensor 59497: model+plot, predictions, hindcast, metadata, saved
[10/103] Sensor 59593: model+plot, predictions, hindcast, metadata, saved
[11/103] Sensor 59650: model+plot, predictions, hindcast, metadata, saved
[12/103] Sensor 59656: mo

In [20]:
# --- DIAGNOSTIC: Check data coverage by year ---
print("\n📊 DATA COVERAGE DIAGNOSTIC")
print("="*60)

for feature_name, df_cached in cached_feature_data.items():
    print(f"\n{feature_name}:")
    
    # Check one sensor as example
    sample_sensor = df_cached["sensor_id"].iloc[0]
    sensor_data = df_cached[df_cached["sensor_id"] == sample_sensor].copy()
    sensor_data["year"] = pd.to_datetime(sensor_data["date"]).dt.year
    
    print(f"  Sensor {sample_sensor}:")
    print(f"  Total rows: {len(sensor_data)}")
    print(f"  Date range: {sensor_data['date'].min()} to {sensor_data['date'].max()}")
    
    # Check pm25 by year
    yearly_stats = sensor_data.groupby("year").agg({
        "pm25": ["count", lambda x: x.isna().sum()]
    })
    yearly_stats.columns = ["Total Rows", "NaN Count"]
    yearly_stats["Non-NaN"] = yearly_stats["Total Rows"] - yearly_stats["NaN Count"]
    print("\n  PM2.5 availability by year:")
    print(yearly_stats)
    
    # After dropna
    sensor_data_clean = sensor_data.dropna(subset=["pm25"])
    if len(sensor_data_clean) > 0:
        print(f"\n  After dropna(subset=['pm25']):")
        print(f"    Remaining rows: {len(sensor_data_clean)}")
        print(f"    Date range: {sensor_data_clean['date'].min()} to {sensor_data_clean['date'].max()}")
        print(f"    Years present: {sorted(sensor_data_clean['year'].unique())}")
    else:
        print(f"\n  ⚠️ NO DATA LEFT after dropna!")
    
    break  # Only check first feature view for now

print("\n" + "="*60)


📊 DATA COVERAGE DIAGNOSTIC

baseline:
  Sensor 60889:
  Total rows: 801
  Date range: 2019-12-09 00:00:00 to 2022-02-21 00:00:00

  PM2.5 availability by year:
      Total Rows  NaN Count  Non-NaN
year                                
2019          23          0       23
2020         366          0      366
2021         360          0      360
2022          52          0       52

  After dropna(subset=['pm25']):
    Remaining rows: 801
    Date range: 2019-12-09 00:00:00 to 2022-02-21 00:00:00
    Years present: [2019, 2020, 2021, 2022]



### 3.6.3. Preparation for Visualization and Registration

In [23]:
df = pd.concat(all_test_data, ignore_index=True) if all_test_data else pd.DataFrame()
df = df.sort_values(by=["date"])
df_by_sensor = {sid: g.copy() for sid, g in df.groupby("sensor_id")}

HINDCAST_MONTHS = 18

## 3.7. Visualization
Generate Plots and Hindcats Data

In [ ]:
all_test_data = []
total_sensors = len(best_models)

print(f"Generating visualizations for {total_sensors} sensors...\n")

for idx, (_, row) in enumerate(best_models.iterrows(), start=1):
    sensor_id = row.name
    best_feature = row["feature_name"]

    status = []

    # Load cached feature view data
    df = cached_feature_data[best_feature]
    df = df[df["sensor_id"] == sensor_id].copy()
    df = df.sort_values("date").reset_index(drop=True)
    df = df.dropna(subset=[TARGET])

    feature_cols = [c for c in df.columns if c not in EXCLUDE_COLS]

    if len(df) < MIN_ROWS:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (too few rows)")
        continue

    # Generate predictions for full dataset
    model_obj = models[best_feature][sensor_id]
    df["predicted_pm25"] = model_obj.predict(df[feature_cols])
    df["best_model"] = best_feature
    status.append("predictions")

    # Hindcast window (last N months)
    cutoff_date = pd.Timestamp.now() - pd.DateOffset(months=HINDCAST_MONTHS)
    df_hindcast = df[df["date"] >= cutoff_date].copy()

    if df_hindcast.empty:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no recent data)")
        continue
    status.append("hindcast")

    # Attach metadata
    if sensor_id in aq_metadata.index:
        df["latitude"] = aq_metadata.at[sensor_id, "latitude"]
        df["longitude"] = aq_metadata.at[sensor_id, "longitude"]
        df["city"] = aq_metadata.at[sensor_id, "city"]
        df["street"] = aq_metadata.at[sensor_id, "street"]
        status.append("metadata")
    else:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no metadata)")
        continue

    # Save plots
    sensor_dir = f"{model_dir}/{sensor_id}"
    images_dir = f"{sensor_dir}/images"
    os.makedirs(images_dir, exist_ok=True)

    # Feature importance plot
    plot_importance(model_obj)
    plt.savefig(f"{images_dir}/feature_importance.png")
    plt.close()
    status.append("feature_importance")

    # Hindcast plot
    fig = visualization.plot_air_quality_forecast(
        df["city"].iloc[0],
        df["street"].iloc[0],
        df_hindcast,
        f"{images_dir}/hindcast_training.png",
        hindcast=True
    )
    if fig is not None:
        fig.suptitle(f"{df['city'].iloc[0]} {df['street'].iloc[0]} (Best Model: {best_feature})")
        plt.close(fig)
    status.append("hindcast_plot")

    # Append data
    all_test_data.append(
        df[[
            "date", "sensor_id", "pm25", "predicted_pm25",
            "latitude", "longitude", "city", "street", "best_model"
        ]]
    )
    status.append("saved")

    print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: " + ", ".join(status))

print(f"\n✅ Visualization complete: {len(all_test_data)} sensors processed.")

Generating visualizations for 103 sensors...

[1/103] Sensor 57421: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[2/103] Sensor 58666: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[3/103] Sensor 58909: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[4/103] Sensor 58912: skipped (no recent data)
[5/103] Sensor 58921: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[6/103] Sensor 59095: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[7/103] Sensor 59356: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[8/103] Sensor 59410: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[9/103] Sensor 59497: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[10/103] Sensor 59593: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[11/103] Sensor 59650: predictions, hindca

## 3.8. Model Registration

In [ ]:
uploaded = 0
total_sensors = len(best_models)

print(f"Registering models for {total_sensors} sensors...\n")

MAX_RETRIES = 5

for sensor_id, row in best_models.iterrows():

    best_feature = row["feature_name"]
    best_r2 = row["R2"]
    best_mse = row["MSE"]

    # Get trained model
    model_obj = models[best_feature][sensor_id]

    # Save model locally
    sensor_model_dir = f"{model_dir}/{sensor_id}"
    os.makedirs(sensor_model_dir, exist_ok=True)
    model_obj.save_model(f"{sensor_model_dir}/model.json")

    # Register model
    model = mr.python.create_model(
        name=f"air_quality_xgboost_model_{sensor_id}",
        metrics={"R2": best_r2, "MSE": best_mse},
        feature_view=feature_views[best_feature],
        description=f"PM2.5 predictor for sensor {sensor_id} using {best_feature} features",
    )

    success = False
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            model.save(sensor_model_dir)
            success = True
            break
        except Exception as e:
            print(f"  ⚠️ Sensor {sensor_id}: attempt {attempt}/{MAX_RETRIES} failed ({type(e).__name__})")
            if attempt < MAX_RETRIES:
                time.sleep(2 * attempt)  # simple backoff
            else:
                print(f"  ❌ Sensor {sensor_id}: giving up after {MAX_RETRIES} attempts")

    if success:
        uploaded += 1
        print(f"[{uploaded}/{total_sensors}] Sensor {sensor_id}: registered ({best_feature})")
    else:
        print(f"[--/--] Sensor {sensor_id}: FAILED to register")

print(f"\nDone. {uploaded}/{total_sensors} models successfully registered.")

Registering models for 103 sensors...

2026-01-26 21:00:34,629 WARNING: ProvenanceWarning: Provenance cached data - feature view provided, but training dataset version is missing

2026-01-26 21:00:34,630 WARNING: ProvenanceWarning: Model schema cannot not be inferred without both the feature view and the training dataset version.



  0%|          | 0/6 [00:00<?, ?it/s]

Uploading c:\Users\krist\Documents\GitHub\pm25/models/57421/model.json: 0.000%|          | 0/455382 elapsed<00…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/57421\images/feature_importance.png: 0.000%|          | …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/57421\images/forecast.png: 0.000%|          | 0/32809 el…

...

Uploading c:\Users\krist\Documents\GitHub\pm25/models/351115\images/hindcast_training.png: 0.000%|          | …

Model created, explore it at https://c.app.hopsworks.ai:443/p/1279184/models/air_quality_xgboost_model_351115/14
[87/103] Sensor 351115: registered (lagged_1d)


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading c:\Users\krist\Documents\GitHub\pm25/models/376954/model.json: 0.000%|          | 0/461007 elapsed<0…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/376954\images/feature_importance.png: 0.000%|          |…

## 3.9. Upload Plots

In [ ]:
dataset_api = project.get_dataset_api()
base_dir = "Resources/plots"
try:
    dataset_api.mkdir(base_dir)
except:
    pass

uploaded_images = 0
total_sensors = len(sensor_locations)

print(f"Uploading plots for {total_sensors} sensors...\n")

for sensor_id in sensor_locations.keys():
    sensor_dir = f"{base_dir}/{sensor_id}"
    try:
        dataset_api.mkdir(sensor_dir)
    except:
        pass

    local_path = f"{model_dir}/{sensor_id}/images/hindcast_training.png"
    remote_path = f"{sensor_dir}/hindcast_training.png"

    ok = hopsworks_admin.safe_upload(dataset_api, local_path, remote_path)

    if ok:
        uploaded_images += 1
        print(f"Uploaded image for sensor {sensor_id} ({uploaded_images}/{total_sensors})")
    else:
        print(f"❌ [fail] Sensor {sensor_id}: upload failed after retries")

print(f"✅ Done uploading {uploaded_images} images.")
